In [ ]:
# xmlrpc_server.ipynb

from SimpleXMLRPCServer import SimpleXMLRPCServer
from SimpleXMLRPCServer import SimpleXMLRPCRequestHandler
import xmlrpclib
import datetime
import pandas as pd
import pickle


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

# Тест
def ping():
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"

server.register_function(black_list_check, 'black_list_check')

# Бинарная передача данных
def send_back_binary(bin_data):
    data = bin_data.data
    return xmlrpclib.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]    
    for i in xrange(height):
        for j in xrange(width):
            img_arr[i][j][0] = 255 - img_arr[i][j][0]
            img_arr[i][j][1] = 255 - img_arr[i][j][1]
            img_arr[i][j][2] = 255 - img_arr[i][j][2]
    
    pimg = pickle.dumps(img_arr)        
    return xmlrpclib.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')


print "Listening on port 8008..."
server.serve_forever()
